This python script uses the pickled best base model. Retrieves data from database automatically, then makes predictions on these live data. At the end, writes these predictions into a results table inside database. 

This script is set to run periodically with the use of Microsoft Task Scheduler. It runs at predetermined times and writes predictions into database. 

** I didn't run this Jupyter file properly because I masked all the critical info about our database. So, database retriaval part did not run as it was in real time. But when I enter the information into dsn and Oracle sections, it runs properly. 

In [1]:
#this version is for real time prediction of updated data; using the pickled base model.


import pandas as pd
import numpy as np
pd.set_option('display.max_columns',40)
import timeit
import warnings
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning

np.random.seed(234)
pd.options.mode.chained_assignment = None

warnings.filterwarnings("ignore", category=DataConversionWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

start_time = timeit.default_timer()

C:\Users\ali.kilinc\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ali.kilinc\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\Users\ali.kilinc\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
#part where csv file is read
#unloadx = pd.read_csv(r'C:\Users\ali.kilinc\Desktop\Roro Boşaltma Tahminleme\Test_Data.csv', index_col= False, encoding= 'unicode_escape')

import pandas as pd
import cx_Oracle
#conn = cx_Oracle.connect("...", "...", "...",encoding = "UTF-8")
dsnStr = cx_Oracle.makedsn("...", "...", "...")
conn = cx_Oracle.connect(user = "...", password = "...", dsn = dsnStr, encoding="UTF-8")
try:
    query = '''
         SELECT * from vw_roro_unloading_input
             '''
    unload = pd.read_sql(con = conn, sql = query)
finally:
    conn.close()
print(unload.head())
import pandas as pd
import numpy as np

DatabaseError: ORA-12545: Connect failed because target host or object does not exist

In [3]:
inp_transform = 0
#unload = unload.iloc[0:1000,:]

unload['POSITION_CODE'] = unload['POSITION_CODE'].astype('str')
#while retrieving data from CSV, leading 0's are missing, this line refills these 0's if needed
unload['POSITION_CODE'][unload['POSITION_CODE'].str[:1].str.isnumeric() == True] = unload['POSITION_CODE'][unload['POSITION_CODE'].str[:1].str.isnumeric() == True].str.zfill(8)

print(unload.info())

unload.dropna(subset=['RIDE_TYPE_ID'], inplace = True)

unload['POSITION_CODE'] = unload['POSITION_CODE'].astype('str')
unload['RIDE_TYPE_ID'] = unload['RIDE_TYPE_ID'].astype('int')
unload['RIDE_TYPE_ID'] = unload['RIDE_TYPE_ID'].astype('category')
unload['RIDE_TYPE_ID'] = unload['RIDE_TYPE_ID'].apply(str)
unload['SERVICE_TYPE'] = unload['SERVICE_TYPE'].astype('object')
unload['ROUTE_MODE'] = unload['ROUTE_MODE'].astype('category')
unload['DOC_TYPE'] = unload['DOC_TYPE'].astype('category')
unload['TRADELANE'] = unload['TRADELANE'].astype('category')
unload['FIRST_UNL'] = unload['FIRST_UNL'].astype('category')
unload['MIN_DUE'] = pd.to_datetime(unload['MIN_DUE'], format = '%d/%m/%Y %H:%M:%S')
#unload['STARTED_DATE'] = pd.to_datetime(unload['STARTED_DATE'], format = '%d/%m/%Y %H:%M:%S')
unload['ARRIVAL_DATE'] = pd.to_datetime(unload['ARRIVAL_DATE'], format = '%d/%m/%Y %H:%M:%S')

unload['UNL_MONTH'] = unload['ARRIVAL_DATE'].dt.month.astype('category')
unload['UNL_MONTH'] = unload['UNL_MONTH'].apply(str)
unload['UNL_DAY'] = (unload['ARRIVAL_DATE'].dt.weekday + 1).astype('category')
unload['UNL_DAY'] = unload['UNL_DAY'].apply(str)
unload['UNL_HOUR'] = unload['ARRIVAL_DATE'].dt.hour.astype('category')

NameError: name 'unload' is not defined

In [4]:
sub_bins = [0, 2, 5, 8, 11, 14, 17, 20, 23]
sub_labels = [1,2,3,4,5,6,7,8]
unload['UNL_HOUR'] = pd.cut(unload['UNL_HOUR'], bins=sub_bins, labels=sub_labels, include_lowest = True)
unload['UNL_HOUR'] = unload['UNL_HOUR'].apply(str)
print(unload['UNL_HOUR'].head(50))


#!!!!!!!! IT IS THE TRANSFORMATION OF CYCLICAL VARIABLES LIKE MONTH AND QUARTER. ONE-HOT IS AN OPTION BUT IT IS MUCH MORE BETTER THAN ENCODING
unload['UNL_MONTH_X'] = np.sin(2*np.pi*unload['UNL_MONTH'].astype(int)/12)
unload['UNL_MONTH_Y'] = np.cos(2*np.pi*unload['UNL_MONTH'].astype(int)/12)

unload['UNL_DAY_X'] = np.sin(2*np.pi*unload['UNL_DAY'].astype(int)/7)
unload['UNL_DAY_Y'] = np.cos(2*np.pi*unload['UNL_DAY'].astype(int)/7)

unload['UNL_HOUR_X'] = np.sin(2*np.pi*unload['UNL_HOUR'].astype(int)/len(sub_labels))
unload['UNL_HOUR_Y'] = np.cos(2*np.pi*unload['UNL_HOUR'].astype(int)/len(sub_labels))

unload.drop(columns = ['UNL_MONTH', 'UNL_DAY', 'UNL_HOUR'], axis = 1, inplace = True)

""" # General function for this transformation
def encode(data, col, max_val): # max val is theoretical maximum of column, for month it is 12, for day of year it is 365 etc.
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data
"""

NameError: name 'unload' is not defined

In [5]:
unload['TIME_DUE'] = (round((unload['MIN_DUE'] - unload['ARRIVAL_DATE']).dt.seconds/60,0)
                            + (unload['MIN_DUE'] - unload['ARRIVAL_DATE']).dt.days*24*60).astype('int')


unload[['T_CUST_TRI', 'TRIESTE_CNT', 'LANECNT', 'TIME_DUE']] = unload[['T_CUST_TRI', 'TRIESTE_CNT', 'LANECNT', 'TIME_DUE']].astype(int)

drop_false_due = unload[(unload['TIME_DUE'] < -30000) | (unload['TIME_DUE'] > 30000)].index
unload.drop(index = drop_false_due, axis = 0, inplace = True)

unload['DOC_TYPE'] = np.where(~unload['DOC_TYPE'].isin(['T-1', 'TR']), 'oth', unload['DOC_TYPE'])
print(unload['DOC_TYPE'].value_counts())
#contains also can be used I think as an alternative
#why it is not working, work on it. For numeric condition like len(x) < 5 it is working.
#unload['DOC_TYPE'] = unload['DOC_TYPE'].replace(unload.groupby('DOC_TYPE').filter(lambda x: unload['DOC_TYPE'].isin(['T-1', 'TR']) == False)['DOC_TYPE'], 'oth')

unload.drop(columns = ['STARTED_DATE'], axis = 1, inplace = True)
# drop=True makes ure that we are not keeping old indexes as a column, Drops that old index column
print(unload.head())
unload.dropna(inplace=True)
unload.reset_index(drop=True, inplace = True)
print(unload.info())

df_pos_code = unload.reset_index()
df_pos_code = df_pos_code[['index', 'POSITION_CODE']]
print(df_pos_code)


df_x = unload.drop(columns = ['ARRIVAL_DATE', 'MIN_DUE', 'POSITION_CODE'], axis=1)

#df_x.drop(columns = ['TRIESTE_CNT', 'T_CUST_TRI', 'SERVICE_TYPE', 'LANECNT', 'TRADELANE', 'DOC_TYPE', 'UNL_HOUR', 'UNL_DAY', 'UNL_MONTH', 'RIDE_TYPE_ID', 'FIRST_UNL'], axis = 1, inplace = True)
df_x.drop(columns = ['TRIESTE_CNT', 'T_CUST_TRI', 'SERVICE_TYPE', 'LANECNT', 'TRADELANE', 'DOC_TYPE'], axis = 1, inplace = True)

df_x = pd.DataFrame(df_x, columns = df_x.columns)
print(df_x.info())

#it is valuable to transform independent variables to decrease skewness etc
if inp_transform == 1:
    tr_cols = ['LANECNT']
    df_x[tr_cols] = np.log1p(df_x[tr_cols])
else:
    pass

NameError: name 'unload' is not defined

In [6]:
import pickle
import numpy as np
import time

with open(r'C:\Users\ali.kilinc\Desktop\Roro Boşaltma Tahminleme\best_model.pkl', 'rb') as f:
        final_model = pickle.load(open('best_model.pkl', 'rb'))

#file = open(r'C:\Users\ali.kilinc\Desktop\Roro Boşaltma Tahminleme\best_model.pkl', 'rb')
#final_model = pickle.load(file)

  
#final_model = pd.read_pickle(r'C:\Users\ali.kilinc\Desktop\Roro Boşaltma Tahminleme\best_model.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'best_model.pkl'

In [7]:
pred = final_model.predict(df_x)
df_x['PRED'] = np.exp(pred)

df_x = pd.merge(df_pos_code, df_x, right_index = True, left_on = 'index')

df_x = df_x.drop(columns = ['index'], axis = 1)

timenow = time.time()
timeobj = time.localtime(timenow)

#it is the flag for each different prediction run
idd = str(timeobj.tm_year) + str(timeobj.tm_mon) + str(timeobj.tm_mday) + str(timeobj.tm_hour) + str(timeobj.tm_min) + str(timeobj.tm_sec)

df_x.insert(0, 'RUN_ID', idd)

print(df_x)
print(df_x.info())

NameError: name 'final_model' is not defined

In [8]:
from sqlalchemy import create_engine

print_excel = df_x.to_excel(r'C:\Users\ali.kilinc\Desktop\PredUnload.xlsx', index = None, header = True, sheet_name = 'FullTable')



dsnStr = cx_Oracle.makedsn("...", "...", "...")
engine = create_engine('oracle+cx_oracle://...')

conn = cx_Oracle.connect(user = "...", password = "...", dsn = dsnStr, encoding="UTF-8")

df_x.to_sql('UNLOAD_PREDICTION_TRIESTE', engine, if_exists='replace')

NameError: name 'df_x' is not defined